In [16]:
import os
from glob import glob
from converter import Converter

In [69]:
raw_dir = '../../raw_videos/google-drive'
repaired_dir = '../../raw_videos/repaired'


input_files = sorted(glob(os.path.join(raw_dir,'*.avi')))

conv = Converter()
for f1 in input_files:
    info = conv.probe(f1)
    if type(info.video.bitrate)==type(None):
        fn,ext = os.path.splitext(os.path.basename(f1))
        f2 = os.path.join(repaired_dir,fn+'-repaired'+ext)
        if not os.path.exists(f2):
            convert = conv.convert(f1, f2, {
                'format':'avi', 
                'video': {'codec':'copy'},
                })
            for progress in convert:
                print(f'{fn}   {100*progress:.1f}%',end='\r')
            print()

Pa_Fri_7dpf_GroupB_n2_CORRUPT   98.7%
Pa_Fri_7dpf_GroupC_n2_CORRUPT   98.6%
Pa_Fri_7dpf_GroupC_n2b_CORRUPT   99.2%
Pa_Fri_7dpf_GroupC_n5_CORRUPT   98.6%
Pa_Sun_14dpf_GroupB_n2c_2020-06-21-114550-0000   90.0%
Pa_Sun_21dpf_GroupC_n1d_2020-06-28-162747-0000   99.3%
Pa_Sun_21dpf_GroupC_n1e_2020-06-28-152755-0000   99.9%
Pa_Sun_21dpf_GroupC_n5_2020-06-28-162753-0000   99.0%
Pa_Sun_7dpf_GroupB_n5a_2020-06-14-135711-0000   99.3%
SF_Sat_28dpf_GroupD_n1a_2020-06-27-175703-0000   99.5%
SF_Sat_28dpf_GroupD_n1b_2020-06-27-175700-0000-020   99.3%
SF_Sat_28dpf_GroupD_n1c_2020-06-27-165707-0000   98.2%
SF_Sat_7dpf_GroupA_n2a_CORRUPT   98.9%
SF_Sat_7dpf_GroupA_n2b_CORRUPT   99.9%
SF_Sat_7dpf_GroupA_n5_CORRUPT   99.9%
SF_Sat_7dpf_GroupB_n2b_CORRUPT___   99.6%
SF_Sat_7dpf_GroupC_n2a_CORRUPT   98.7%
SF_Sat_7dpf_GroupC_n2c_CORRUPT   99.1%
SF_Sat_7dpf_GroupC_n5_CORRUPT   99.7%
SF_Sat_7dpf_GroupD_n2b_CORRUPT   98.5%
SF_Sat_7dpf_GroupD_n2b_CORRUPT_   99.4%
SF_Sat_7dpf_GroupD_n5_CORRUPT   99.5%
